In [28]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
from numpy import array

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import os

In [30]:
data = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/hotel_review_75000.csv')

In [31]:
data.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'hotelId', 'hotelName', 'tripType',
       'tripTypeText', 'reviewDate', 'rating', 'description', 'isKorNot'],
      dtype='object')

In [32]:
data = data[['rating', 'description']]

In [33]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   rating       75000 non-null  float64
 1   description  75000 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


* rating 8 이상이면 긍정 1, 미만은 부정 0 으로 타겟 컬럼을 새로 하나 만들기.
* description컬럼을 토크나이징, 시퀀스로 변환, 임베딩
* train, valid, test로 데이터 분할
* 딥러닝 네트워크 생성 후 분석
* early_stopping, modelcheckpoint활용해서 분석
* 그래프로 loss 출력

In [34]:
def sep_rating(x):
    if x >= 8: 
        return 1
    else:
        return 0

In [35]:
# rating 8 이상이면 긍정 1, 미만은 부정 0 으로 타겟 컬럼을 새로 하나 만들기.
data['sep_rating'] = data['rating'].apply(sep_rating)

In [36]:
data.head()

,rating,description,sep_rating
0,10.0,직원들 서비스나 마인드 완벽합니다 다만 노후된 시설과 좁은 라운지가 호텔 이름에 비...,1
1,10.0,웨스틴조선은 모든면에서 최고의 숙소라고 생각합니다 분위기 편의성 직원친절도까지 모두...,1
2,10.0,서울시내에서 호캉스를 누릴 수 있는 곳아침 뷔페가 맛있는 곳룸서비스가 좋았음요구사항...,1
3,10.0,가격에 맞게 만족했습니다,1
4,10.0,처음 세명으로 셋팅 되어있지 않은점 빼고는 직원들의 친절도 및 서비스는 모두 맘에 ...,1


In [37]:
data['description']

0        직원들 서비스나 마인드 완벽합니다 다만 노후된 시설과 좁은 라운지가 호텔 이름에 비...
1        웨스틴조선은 모든면에서 최고의 숙소라고 생각합니다 분위기 편의성 직원친절도까지 모두...
2        서울시내에서 호캉스를 누릴 수 있는 곳아침 뷔페가 맛있는 곳룸서비스가 좋았음요구사항...
3                                            가격에 맞게 만족했습니다
4        처음 세명으로 셋팅 되어있지 않은점 빼고는 직원들의 친절도 및 서비스는 모두 맘에 ...
                               ...                        
74995    조식과 스탠더드 하프전망 신청해서  1박 했는데요ᆢ너무 좋아요ᆢ다만 가격이 좀 나가...
74996    객실은 깔끔하고 에메니티도 딱 필요한만큼 있었습니다 5분거리에 중앙시장이 있어 쇼핑...
74997    배드가 좋고 조용해서 좋았습니다찬바람이 좀 들어와서 난방기를 12시간 정도 돌려야 ...
74998    우선 위치가 너무 좋았고 숙소에 짐을 놓고 걸어서 주변 가볼곳도 있고 먹거리도 맛있...
74999    주차장이 부족해서 다른데에 주차했는데 아침에 이동해달라는 전화가 와서 다소 불쾌했슴...
Name: description, Length: 75000, dtype: object

In [38]:
data['description'][0]

'직원들 서비스나 마인드 완벽합니다 다만 노후된 시설과 좁은 라운지가 호텔 이름에 비해 약간 부족합니다'

In [49]:
# description컬럼을 토크나이징, 시퀀스로 변환(단어를 벡터로 변환), 임베딩
text = data['description']
token =Tokenizer(lower=False)
token.fit_on_texts(text)
print(token.word_index)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
# 단어를 벡터로 변환(시퀀싱)
x = token.texts_to_sequences(text)

In [51]:
x

[[268, 1162, 5870, 5689, 26, 1875, 661, 581, 4887, 10, 4635, 101, 83, 2604],
 [67077, 1725, 308, 4636, 488, 566, 6066, 67078, 31, 3858],
 [21641,
  1628,
  8335,
  7,
  17,
  42175,
  3242,
  849,
  67079,
  67080,
  825,
  1600,
  21642,
  67081,
  42,
  723,
  67082,
  665,
  4111],
 [117, 2276, 178],
 [214,
  42176,
  12702,
  2929,
  21643,
  224,
  326,
  1108,
  55,
  239,
  31,
  173,
  734,
  77,
  13807,
  1576,
  1485,
  916,
  42177,
  55,
  42178,
  825,
  936,
  981,
  1,
  613,
  1040,
  142],
 [60, 67083, 1941, 1377, 236, 31649, 26, 3936, 6067, 21644, 133, 244, 42179],
 [308, 97, 15, 60, 517, 2, 404, 68, 12, 102],
 [1012, 88, 21645, 143, 20, 5, 1415],
 [71, 47, 2036, 1, 4888, 42180, 42181, 54, 2077],
 [42182,
  9732,
  15124,
  13808,
  394,
  25559,
  134,
  4,
  4311,
  25560,
  1640,
  1317,
  31,
  132],
 [3, 67, 1, 20, 21646, 518],
 [360, 7676, 8, 306, 97, 268, 13, 8, 8, 8, 351, 68, 98, 2198, 4215],
 [13809, 872, 57, 107, 2, 832, 156],
 [67084,
  31650,
  21,
  31,


In [52]:
# 인덱스에 1을 추가해서 원-핫 인코딩 배열 만들기, 
word_size = len(token.word_index) + 1 

In [53]:
word_size

261894

In [64]:
# x = to_categorical(x, num_classes=word_size) 전체를 넣을 때는 카테고리화를 안 해.
#Embedding으로 차원축소

In [56]:
# 긍정 리뷰는 1, 부정리뷰는 0으로 클래스 지정
classes = data['sep_rating']

In [57]:
# chatGPT에 물어본 가장 긴 리스트 길이 찾는 법.
max_len = max (len(i) for i in x)
print("가장 긴 하위 리스트의 길이: ", max_len)

가장 긴 하위 리스트의 길이:  377


In [58]:
# 패딩 추가, pad_sequences(데이터, 가장 긴 문장의 길이)
padded_x = pad_sequences(x, 377)
print("패딩 결과", padded_x)

패딩 결과 [[     0      0      0 ...    101     83   2604]
 [     0      0      0 ...  67078     31   3858]
 [     0      0      0 ...  67082    665   4111]
 ...
 [     0      0      0 ...    195  42071  39796]
 [     0      0      0 ... 261891    502    437]
 [     0      0      0 ...   3199   8900     64]]


X 독립변수로 padded_x가 들어가고, y로는 classes가 들어가게 됨.,

In [60]:
# train, valid, test 데이터 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_x, classes, test_size=0.4, random_state=7)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=7)

In [61]:
# 단어 임베딩을 포함해서 딥러닝 모델 생성
model = Sequential()
model.add(Embedding(word_size, 32, input_length=377))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 377, 32)           8380608   
                                                                 
 flatten (Flatten)           (None, 12064)             0         
                                                                 
 dense (Dense)               (None, 1)                 12065     
                                                                 
Total params: 8,392,673
Trainable params: 8,392,673
Non-trainable params: 0
_________________________________________________________________


In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [63]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)
modelpath="./model/hotels_reviews.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0,
                               save_best_only=True)
history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid), 
                    verbose=1, epochs=5, batch_size=32,
                   callbacks=[early_stopping_callback, checkpointer])
pred = model.predict(X_valid)

Epoch 1/5
1407/1407 [==============================] - 223s 157ms/step - loss: 0.3636 - accuracy: 0.8401 - val_loss: 0.3092 - val_accuracy: 0.8720
Epoch 2/5
1407/1407 [==============================] - 212s 151ms/step - loss: 0.1789 - accuracy: 0.9343 - val_loss: 0.3351 - val_accuracy: 0.8646
Epoch 3/5
1407/1407 [==============================] - 213s 151ms/step - loss: 0.0764 - accuracy: 0.9768 - val_loss: 0.3878 - val_accuracy: 0.8569
Epoch 4/5
1407/1407 [==============================] - 207s 147ms/step - loss: 0.0383 - accuracy: 0.9895 - val_loss: 0.4540 - val_accuracy: 0.8517
Epoch 5/5
469/469 [==============================] - 3s 6ms/step
